# Data Cleaning – Notebook

**Uwaga**  
Ten notatnik pełni rolę *„brudnopisu”* – pokazuje proces czyszczenia danych krok po kroku.  
Kod i wizualizacje w tym notebooku mają charakter roboczy i nie są dopracowane pod względem estetycznym.  

Estetyczne i finalne wersje wyników znajdują się w:  
- `scripts/data_cleaning.py` - czysty, uporządkowany kod,  
- `reports/cleaning_report.md` - finalny raport z czyszczenia danych.  

Celem tego notatnika jest **pokazanie procesu przygotowania danych do analizy**, w tym:  
- obsługa braków danych,  
- uporządkowanie formatów,  
- zapis plików wynikowych w formacie CSV (`data/clean/`),  
- dokumentacja decyzji czyszczących.  

## Importy bibliotek i konfiguracja połączenia

Importujemy niezbędne biblioteki:
- `os` – operacje na plikach i ścieżkach,
- `pandas` – praca z DataFrame,
- `dotenv` – wczytywanie zmiennych środowiskowych,
- `sqlalchemy` – połączenie z bazą PostgreSQL i wykonywanie zapytań.

In [6]:
import os
import pandas as pd
from dotenv import load_dotenv
from sqlalchemy import create_engine, text

## Wczytanie zmiennych z pliku `.env`

Ładujemy dane do logowania do bazy z pliku `.env`:
- DB_HOST - adres hosta bazy,
- DB_NAME - nazwa bazy danych,
- DB_USER - użytkownik,
- DB_PASS - hasło.

In [7]:
load_dotenv()

DB_HOST = os.getenv("DB_HOST")
DB_NAME = os.getenv("DB_NAME")
DB_USER = os.getenv("DB_USER")
DB_PASS = os.getenv("DB_PASS")

## Połączenie z bazą danych

Tworzymy połączenie do bazy PostgreSQL z wykorzystaniem SQLAlchemy.
Sprawdzamy również wersję bazy, aby upewnić się, że połączenie działa.

In [8]:
engine = create_engine(f"postgresql+psycopg2://{DB_USER}:{DB_PASS}@{DB_HOST}/{DB_NAME}")

with engine.connect() as conn:
    version = conn.execute(text("SELECT version()")).scalar()
    print(f"Połączono z bazą. Wersja PostgreSQL: {version}")

Połączono z bazą. Wersja PostgreSQL: PostgreSQL 17.5 on x86_64-windows, compiled by msvc-19.44.35209, 64-bit


## Lista tabel do czyszczenia

Definuijemy, które tabele chcemy oczyścić w bazie.

In [9]:
tables = ["orders", "order_items", "sessions", "events", "users", "sellers", "products"]

## Czyszczenie tabel

Dla każdej tabeli wykonujemy:
1. Wczytanie danych do Pandas DataFrame.
2. Specyficzne poprawki dla wybranych tabel (np. brakujące kolumny, błędne nazwy).
3. Ujednolicenie dat.
4. Zapis wyczyszczonych danych do schematu `clean`.

In [ ]:
for table in tables:
    print(f"\n=== Czyszczenie tabeli: {table} ===")
    df = pd.read_sql(f'SELECT * FROM "{table}"', engine)

    # -----------------------
    # Specyficzne poprawki
    # -----------------------
    
    if table == "sellers":
        bad_cols = [col for col in df.columns if ";" in col]
        df.drop(columns=bad_cols, inplace=True, errors="ignore")
        if "location" in df.columns:
            df["location"].fillna("unknown", inplace=True)

    if table == "products":
        if "stock_quantity" in df.columns:
            df["stock_quantity"].fillna(0, inplace=True)

    if table == "events":
        for col in ["product_id", "seller_id"]:
            if col in df.columns:
                df[col].fillna(-1, inplace=True)

    # -----------------------
    # Ujednolicenie dat
    # -----------------------
    for col in df.columns:
        if "date" in col or "time" in col:
            try:
                df[col] = pd.to_datetime(df[col])
            except Exception:
                pass

    # -----------------------
    # Zapis do schematu "clean"
    # -----------------------
    df.to_sql(table, engine, schema="clean", if_exists="replace", index=False)
    print(f"✔ Tabela '{table}' zapisana ({len(df)} wierszy).")

print("\n✅ Czyszczenie zakończone!")


=== Czyszczenie tabeli: orders ===
✔ Tabela 'orders' zapisana (80000 wierszy).

=== Czyszczenie tabeli: order_items ===
✔ Tabela 'order_items' zapisana (219702 wierszy).

=== Czyszczenie tabeli: sessions ===
✔ Tabela 'sessions' zapisana (100000 wierszy).

=== Czyszczenie tabeli: events ===


C:\Users\Michał\AppData\Local\Temp\ipykernel_560\208264470.py:21: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(-1, inplace=True)


✔ Tabela 'events' zapisana (200000 wierszy).

=== Czyszczenie tabeli: users ===
✔ Tabela 'users' zapisana (10000 wierszy).

=== Czyszczenie tabeli: sellers ===
✔ Tabela 'sellers' zapisana (75 wierszy).

=== Czyszczenie tabeli: products ===
✔ Tabela 'products' zapisana (2000 wierszy).

✅ Czyszczenie zakończone!


C:\Users\Michał\AppData\Local\Temp\ipykernel_560\208264470.py:12: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["location"].fillna("unknown", inplace=True)
C:\Users\Michał\AppData\Local\Temp\ipykernel_560\208264470.py:16: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example

## Podsumowanie

- Wszystkie tabele zostały oczyszczone i zapisane w schemacie `clean`.
- Braki danych zostały uzupełnione.
- Kolumny z datami są w formacie datetime.
- Tabele gotowe do dalszej analizy w SQL lub Power BI